## Imports and Config

In [1]:
!pip install ccxt
!pip install alpaca-trade-api
!pip install fredapi

     |████████████████████████████████| 2.5 MB 18.1 MB/s 
     |████████████████████████████████| 271 kB 45.4 MB/s 
     |████████████████████████████████| 1.1 MB 26.0 MB/s 
     |████████████████████████████████| 3.6 MB 47.7 MB/s 
     |████████████████████████████████| 1.1 MB 38.5 MB/s 
     |████████████████████████████████| 94 kB 1.9 MB/s 
     |████████████████████████████████| 291 kB 11.7 MB/s 
     |████████████████████████████████| 144 kB 45.6 MB/s 
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.4.0
    Uninstalling setuptools-57.4.0:
      Successfully uninstalled setuptools-57.4.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


     |████████████████████████████████| 54 kB 2.7 MB/s 
     |████████████████████████████████| 111 kB 25.0 MB/s 
     |████████████████████████████████| 596 kB 56.5 MB/s 
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import ccxt
from fredapi import Fred
from alpaca_trade_api.rest import REST, TimeFrame, TimeFrameUnit
from alpaca_trade_api.common import URL

import numpy as np
import pandas as pd

import pickle
from datetime import datetime, timedelta

from sklearn.decomposition import PCA

## Load CSV File

In [ ]:
# dateparse = lambda dates: [datetime.strptime(d, '%Y-%m-%d %H:%M:%S') for d in dates]

In [22]:
df = pd.read_csv('IS5006_Historical.csv', index_col='datetime', parse_dates=[0], dayfirst=True)
df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR,Open,High,Low,Close,Volume,Bollinger
datetime,,,,,,,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111,29413.29,29683.50,29039.00,29683.03,4431.837894,0
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546,32222.88,34205.00,32008.62,33779.48,7955.017558,-1
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450,33082.83,33666.99,32453.37,32903.87,7401.195215,0
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187,32020.22,32890.00,30020.00,30460.19,11908.475812,0
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698,34043.91,35848.99,33352.54,35130.05,11648.279116,-1


In [23]:
df.tail()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR,Open,High,Low,Close,Volume,Bollinger
datetime,,,,,,,,,,,,
2022-02-20,1,334.488761,24.074482,0.790679,0.0,4203,39370.68,39506.91,38000.0,39248.58,13482.777128,0
2022-02-21,0,455.129095,24.827410,0.645460,-1.0,3926,39248.56,39383.08,36350.0,36751.36,22849.665589,0
2022-02-22,0,575.769429,25.580338,0.500242,0.0,4066,36753.62,38469.06,36534.6,37996.82,19736.071241,0
2022-02-23,0,1040.616450,28.084669,1.338296,0.0,3696,38000.42,39276.85,34322.0,34644.77,27100.305347,1
2022-02-24,0,948.544133,28.137896,0.788475,0.0,4110,34662.32,39720.00,34357.1,38467.77,31567.220250,0


## Get Historical Prices

In [7]:
exch = 'hitbtc'
period = '1d'
start = datetime(2020, 1, 1) # Earlier date to facilitate moving average calculation
end = datetime(2022, 3, 1)
limit = 500
days = (end - start).days
symbol = 'BTCUSD'

In [8]:
def convert_gmt_to_local(t):
    return(t + (datetime.now() - datetime.utcnow()))

In [9]:
historical_df = pd.DataFrame(columns=['Open', 'High', 'Low', 'Close', 'Volume'])
url = URL('https://paper-api.alpaca.markets')
api = REST(key_id='PKOZW3IUKPOP2RAGNB1E', secret_key='GXFPg7RmGX5xp0pIL0jnDKHj6UYBoEWSAmJlDj51', base_url=url)
for i in range(int(days//limit)+1):
    since = ((start+timedelta(limit*i))).strftime('%Y-%m-%d')
    ohlcv = api.get_crypto_bars('BTCUSD', TimeFrame(1, TimeFrameUnit.Day), since, None, limit, ['CBSE']).df
    ohlcv.index = pd.to_datetime(convert_gmt_to_local(ohlcv.index)).date
    ohlcv.drop(['exchange', 'trade_count', 'vwap'], axis=1, inplace=True)
    ohlcv.columns=['Open', 'High', 'Low', 'Close', 'Volume']
    ohlcv.index.rename('Timestamp', inplace=True)
    historical_df = historical_df.append(ohlcv)
historical_df.head()

,Open,High,Low,Close,Volume
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611


In [10]:
historical_df.tail()

,Open,High,Low,Close,Volume
2022-03-26,44433.22,44973.50,44156.22,44601.24,7235.396772
2022-03-27,44598.44,47694.00,44437.22,47148.98,12278.601793
2022-03-28,47149.30,48240.00,46770.70,47624.51,17868.795255
2022-03-29,47624.33,48124.94,46544.89,47255.62,15656.635480
2022-03-30,47258.85,47717.01,46834.12,47174.55,11327.240732


In [11]:
historical_df.index = pd.to_datetime(historical_df.index, errors='coerce')

In [ ]:
updated_df = df.merge(historical_df, left_on=df.index, right_on=historical_df.index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR,Bollinger,Open_x,High_x,Low_x,Close_x,Volume_x,Open_y,High_y,Low_y,Close_y,Volume_y
datetime,,,,,,,,,,,,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111,0.0,29413.29,29683.50,29039.00,29683.03,4431.837894,29413.29,29683.50,29039.00,29683.03,4431.837894
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546,1.0,32222.88,34205.00,32008.62,33779.48,7955.017558,32222.88,34205.00,32008.62,33779.48,7955.017558
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450,-1.0,33082.83,33666.99,32453.37,32903.87,7401.195215,33082.83,33666.99,32453.37,32903.87,7401.195215
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187,0.0,32020.22,32890.00,30020.00,30460.19,11908.475812,32020.22,32890.00,30020.00,30460.19,11908.475812
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698,1.0,34043.91,35848.99,33352.54,35130.05,11648.279116,34043.91,35848.99,33352.54,35130.05,11648.279116


In [ ]:
updated_df.to_csv('IS5006_Historical.csv')

## Add MA Crossover

In [ ]:
# df.drop(['MA_Signal'], axis=1, inplace=True)
# df.head()

In [ ]:
ma_df = historical_df.copy(deep = True)

In [ ]:
ma_df['EMA_10'] = ma_df['Close'].ewm(span=10, adjust = False).mean()
ma_df['SMA_50'] = ma_df['Close'].rolling(50).mean()
ma_df[f'MA_Position'] = np.where(ma_df[f'EMA_10'] > ma_df[f'SMA_50'], 1.0, 0.0)
ma_df[f'MA_Signal'] = ma_df[f'MA_Position'].diff()
ma_df.drop([f'MA_Position'], axis=1, inplace=True)
ma_df.index = pd.to_datetime(ma_df.index, errors='coerce')
ma_df.head()

,Open,High,Low,Close,Volume,EMA_10,SMA_50,MA_Signal
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568,7108.030000,NaN,NaN
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853,7125.304545,NaN,0.0
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578,7163.330992,NaN,0.0
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227,7197.021721,NaN,0.0
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611,7257.326862,NaN,0.0


In [ ]:
ma_df['MA_Signal'].value_counts()

 0.0    798
 1.0     11
-1.0     10
Name: MA_Signal, dtype: int64

In [ ]:
updated_df = df.merge(ma_df[['MA_Signal']], left_on=df.index, right_on=ma_df[['MA_Signal']].index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal_x,VaR,Bollinger,Open,High,Low,Close,Volume,MA_Signal_y
datetime,,,,,,,,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111,0.0,29413.29,29683.50,29039.00,29683.03,4431.837894,0.0
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546,1.0,32222.88,34205.00,32008.62,33779.48,7955.017558,0.0
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450,-1.0,33082.83,33666.99,32453.37,32903.87,7401.195215,0.0
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187,0.0,32020.22,32890.00,30020.00,30460.19,11908.475812,0.0
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698,1.0,34043.91,35848.99,33352.54,35130.05,11648.279116,0.0


In [ ]:
updated_df.to_csv('IS5006_Historical.csv')

## Add Macroeconomic Data

In [ ]:
macro_series = {'inflation': 'FPCPITOTLZGUSA', 'unemployment': 'UNRATE', 
        'oil': 'DCOILWTICO', 'gold_vix': 'GVZCLS', 'cboe_vix':'VIXCLS', 'dow_jones': 'DJIA',
        'interest_rate': 'IR3TIB01USM156N'}

In [ ]:
fred = Fred(api_key='40ed0248dfecd2195c2bc8353707eead')

In [ ]:
macro_df = pd.DataFrame(index=df.index)
for key in macro_series.keys():
  temp_df = fred.get_series(macro_series[key])
  temp_df.name = key
  macro_df = macro_df.merge(temp_df, left_on=macro_df.index, right_on=temp_df.index, how='outer')
  macro_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
  macro_df.set_index('datetime', inplace=True)
  macro_df.sort_index(ascending=True, inplace=True)

In [ ]:
macro_df.tail()

,inflation,unemployment,oil,gold_vix,cboe_vix,dow_jones,interest_rate
datetime,,,,,,,
2022-03-24,NaN,NaN,114.20,24.07,21.67,34707.94,NaN
2022-03-25,NaN,NaN,116.20,24.68,20.81,34861.24,NaN
2022-03-28,NaN,NaN,107.55,23.20,19.63,34955.89,NaN
2022-03-29,NaN,NaN,NaN,21.28,18.90,35294.19,NaN
2022-03-30,NaN,NaN,NaN,NaN,NaN,35228.81,NaN


In [ ]:
macro_df['inflation'] = macro_df['inflation'].fillna(method="ffill")
macro_df['unemployment'] = macro_df['unemployment'].fillna(method="ffill")
macro_df['oil'] = macro_df['oil'].interpolate(method='time')
macro_df['gold_vix'] = macro_df['gold_vix'].interpolate(method='time')
macro_df['cboe_vix'] = macro_df['cboe_vix'].interpolate(method='time')
macro_df['dow_jones'] = macro_df['dow_jones'].interpolate(method='time')
macro_df['interest_rate'] = macro_df['interest_rate'].fillna(method="ffill")
macro_df.tail()

,inflation,unemployment,oil,gold_vix,cboe_vix,dow_jones,interest_rate
datetime,,,,,,,
2022-03-24,1.233584,3.8,114.20,24.07,21.67,34707.94,0.38
2022-03-25,1.233584,3.8,116.20,24.68,20.81,34861.24,0.38
2022-03-28,1.233584,3.8,107.55,23.20,19.63,34955.89,0.38
2022-03-29,1.233584,3.8,107.55,21.28,18.90,35294.19,0.38
2022-03-30,1.233584,3.8,107.55,21.28,18.90,35228.81,0.38


In [ ]:
# df.drop(['inflation',	'unemployment',	'oil',	'gold_vol'], axis=1, inplace=True)

In [ ]:
updated_df = df.merge(macro_df, left_on=df.index, right_on=macro_df.index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
# updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR,Bollinger,Open,High,Low,Close,Volume,inflation,unemployment,oil,gold_vix,cboe_vix,dow_jones,interest_rate
datetime,,,,,,,,,,,,,,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111,0.0,29413.29,29683.50,29039.00,29683.03,4431.837894,1.233584,6.4,48.13,21.0575,23.805,30510.8325,0.14
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546,1.0,32222.88,34205.00,32008.62,33779.48,7955.017558,1.233584,6.4,47.91,21.3250,24.860,30415.1850,0.14
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450,-1.0,33082.83,33666.99,32453.37,32903.87,7401.195215,1.233584,6.4,47.69,21.5925,25.915,30319.5375,0.14
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187,0.0,32020.22,32890.00,30020.00,30460.19,11908.475812,1.233584,6.4,47.47,21.8600,26.970,30223.8900,0.14
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698,1.0,34043.91,35848.99,33352.54,35130.05,11648.279116,1.233584,6.4,49.78,22.1400,25.340,30391.6000,0.14


In [ ]:
# pca = PCA(n_components=3)
# pca.fit(updated_df[list(macro_series.keys())])
# print(pca.explained_variance_ratio_)
# with open('macro_pca.pkl','wb') as f:
#     pickle.dump(pca,f)

In [ ]:
pca = None
with open('macro_pca.pkl', 'rb') as f:
    pca = pickle.load(f)

In [ ]:
pca_cols = pd.DataFrame(pca.transform(updated_df[list(macro_series.keys())]), columns = [f'MACRO_{i}' for i in range(pca.n_components)], index=updated_df.index)
pca_cols.head()

,MACRO_0,MACRO_1,MACRO_2
datetime,,,
2021-01-01,3661.709111,-5.580458,0.925199
2021-01-02,3757.358041,-5.094828,1.771221
2021-01-03,3853.006971,-4.609197,2.617242
2021-01-04,3948.655902,-4.123567,3.463264
2021-01-05,3780.935518,-3.040439,1.718721


In [ ]:
updated_df.drop(list(macro_series.keys()), axis=1, inplace=True)
updated_df = updated_df.merge(pca_cols, left_on=updated_df.index, right_on=pca_cols.index)
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MA_Signal,MACRO_0,MACRO_1,MACRO_2
datetime,,,,,
2021-01-01,0,0,3661.709111,-5.580458,0.925199
2021-01-02,0,0,3757.358041,-5.094828,1.771221
2021-01-03,0,0,3853.006971,-4.609197,2.617242
2021-01-04,0,0,3948.655902,-4.123567,3.463264
2021-01-05,0,0,3780.935518,-3.040439,1.718721


In [ ]:
updated_df.tail()

,Fuzzy_sentiment_signal,MA_Signal,MACRO_0,MACRO_1,MACRO_2
datetime,,,,,
2022-02-20,1,-1,334.488761,24.074482,0.790679
2022-02-21,0,0,455.129095,24.827410,0.645460
2022-02-22,0,0,575.769429,25.580338,0.500242
2022-02-23,0,0,1040.616450,28.084669,1.338296
2022-02-24,0,0,948.544133,28.137896,0.788475


In [ ]:
updated_df.index = pd.to_datetime(updated_df.index)
updated_df.to_csv('IS5006_Historical.csv')

## Add VaR Code

In [ ]:
var_df = historical_df.copy(deep = True)
var_df

,Open,High,Low,Close,Volume
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611
...,...,...,...,...,...
2022-03-17,41360.67,41486.00,40261.58,40646.56,15356.127485
2022-03-18,40645.27,42396.04,40130.00,41824.46,16287.481907
2022-03-19,41823.30,42429.00,41560.49,41875.48,9292.766254
2022-03-20,41875.01,42037.81,40509.00,40882.31,7447.710197


In [ ]:
def calc_var(price, df):
  periodic_ret = df['Close'].pct_change().dropna().sort_values().reset_index(drop=True)
  xth = int(np.floor(0.01*len(periodic_ret))) - 1
  xth_smallest_rate = periodic_ret[xth]
  mean_return_rate = periodic_ret.mean()
  VaR = price * (mean_return_rate - xth_smallest_rate)
  return(VaR)

In [ ]:
new_var_df = var_df.iloc[366:]
new_var_df['VaR'] = 0
for ix, row in new_var_df.iterrows():
  old_df = var_df[(ix-timedelta(days=300)): ix]
  new_var_df.at[ix, 'VaR'] = calc_var(row['Close'], old_df)
new_var_df.index = pd.to_datetime(new_var_df.index, errors='coerce')
new_var_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Open,High,Low,Close,Volume,VaR
2021-01-01,29413.29,29683.50,29039.00,29683.03,4431.837894,3111
2021-01-02,32222.88,34205.00,32008.62,33779.48,7955.017558,3546
2021-01-03,33082.83,33666.99,32453.37,32903.87,7401.195215,3450
2021-01-04,32020.22,32890.00,30020.00,30460.19,11908.475812,3187
2021-01-05,34043.91,35848.99,33352.54,35130.05,11648.279116,3698


In [ ]:
updated_df = df.merge(new_var_df[['VaR']], left_on=df.index, right_on=new_var_df[['VaR']].index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR
datetime,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698


In [ ]:
updated_df.to_csv('IS5006_Historical.csv')

## Bollinger Agent

In [17]:
bol_df = historical_df.copy(deep = True)
bol_df.head()

,Open,High,Low,Close,Volume
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611


In [18]:
bol_df['SMA_20'] = bol_df['Close'].rolling(20).mean()
bol_df['STD_20'] = bol_df['Close'].rolling(20).std()
bol_df['LOW_Band_20'] = bol_df['SMA_20'] - bol_df['STD_20'] * 2
bol_df['HIGH_Band_20'] = bol_df['SMA_20'] + bol_df['STD_20'] * 2
bol_df['Sell_Signal'] = np.where(bol_df['Close'] >= bol_df['HIGH_Band_20'], -1, 0)
bol_df['Sell_Position'] = bol_df['Sell_Signal'].diff()
bol_df['Sell_Position'] = np.where(bol_df['Sell_Position'] == -1, -1, 0)
bol_df['Buy_Signal'] = np.where(bol_df['Close'] <= bol_df['LOW_Band_20'], 1, 0)
bol_df['Buy_Position'] = bol_df['Buy_Signal'].diff()
bol_df['Buy_Position'] = np.where(bol_df['Buy_Position'] == 1, 1, 0)
bol_df['Bollinger'] = bol_df['Sell_Position'] + bol_df['Buy_Position']
bol_df.index = pd.to_datetime(bol_df.index, errors='coerce')
bol_df.head()

,Open,High,Low,Close,Volume,SMA_20,STD_20,LOW_Band_20,HIGH_Band_20,Sell_Signal,Sell_Position,Buy_Signal,Buy_Position,Bollinger
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611,NaN,NaN,NaN,NaN,0,0,0,0,0


In [19]:
# df.drop(['Bollinger'], axis=1, inplace=True)

In [20]:
updated_df = df.merge(bol_df[['Bollinger']], left_on=df.index, right_on=bol_df[['Bollinger']].index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR,Open,High,Low,Close,Volume,Bollinger
datetime,,,,,,,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111,29413.29,29683.50,29039.00,29683.03,4431.837894,0
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546,32222.88,34205.00,32008.62,33779.48,7955.017558,-1
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450,33082.83,33666.99,32453.37,32903.87,7401.195215,0
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187,32020.22,32890.00,30020.00,30460.19,11908.475812,0
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698,34043.91,35848.99,33352.54,35130.05,11648.279116,-1


In [21]:
updated_df.to_csv('IS5006_Historical.csv')

# RSI

In [24]:
rsi_df = historical_df.copy(deep = True)
rsi_df

,Open,High,Low,Close,Volume
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611
...,...,...,...,...,...
2022-03-26,44433.22,44973.50,44156.22,44601.24,7235.396772
2022-03-27,44598.44,47694.00,44437.22,47148.98,12278.601793
2022-03-28,47149.30,48240.00,46770.70,47624.51,17868.795255
2022-03-29,47624.33,48124.94,46544.89,47255.62,15656.635480


In [25]:
#Calculate 14 day RSI value, ranges betweeen 0 to 100
rsi_df['diff'] = rsi_df['Close'].diff(1)
rsi_df['gain'] = rsi_df['diff'].clip(lower=0).round(2)
rsi_df['loss'] = rsi_df['diff'].clip(upper=0).round(2)
rsi_df['avg_gain'] = rsi_df['gain'].rolling(14).mean()
rsi_df['avg_loss'] = -rsi_df['loss'].rolling(14).mean()
rsi_df['rs'] = rsi_df['avg_gain'] / rsi_df['avg_loss']
rsi_df['rsi'] = 100 - (100/(1.0 +rsi_df['rs']))
rsi_df

,Open,High,Low,Close,Volume,diff,gain,loss,avg_gain,avg_loss,rs,rsi
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853,95.01,95.01,0.00,NaN,NaN,NaN,NaN
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578,131.41,131.41,0.00,NaN,NaN,NaN,NaN
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227,14.18,14.18,0.00,NaN,NaN,NaN,NaN
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611,180.07,180.07,0.00,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-26,44433.22,44973.50,44156.22,44601.24,7235.396772,167.92,167.92,0.00,629.586429,238.700000,2.637564,72.509072
2022-03-27,44598.44,47694.00,44437.22,47148.98,12278.601793,2547.74,2547.74,0.00,811.567857,196.307143,4.134174,80.522670
2022-03-28,47149.30,48240.00,46770.70,47624.51,17868.795255,475.53,475.53,0.00,827.055000,196.307143,4.213066,80.817432
2022-03-29,47624.33,48124.94,46544.89,47255.62,15656.635480,-368.89,0.00,-368.89,803.036429,222.656429,3.606617,78.292095


In [30]:
#rsi sell signal: overbought when rsi is more than 70
#rsi buy signal: oversold when rsi is below 30
rsi_overbought = 70
rsi_oversold = 30
rsi_df['Sell_Signal'] = np.where(rsi_df['rsi'] >= rsi_overbought, -1, 0)
rsi_df['Sell_Position'] = rsi_df['Sell_Signal'].diff()
rsi_df['Sell_Position'] = np.where(rsi_df['Sell_Position'] == -1, -1, 0)
rsi_df['Buy_Signal'] = np.where(rsi_df['rsi'] <= rsi_oversold, 1, 0)
rsi_df['Buy_Position'] = rsi_df['Buy_Signal'].diff()
rsi_df['Buy_Position'] = np.where(rsi_df['Buy_Position'] == 1, 1, 0)
rsi_df['RSI_Signal'] = rsi_df['Sell_Position'] + rsi_df['Buy_Position']
rsi_df

,Open,High,Low,Close,Volume,diff,gain,loss,avg_gain,avg_loss,rs,rsi,Sell_Signal,Sell_Position,Buy_Signal,Buy_Position,RSI_Signal
2020-01-01,7174.33,7186.18,7089.00,7108.03,1357.688568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-02,6945.02,7281.24,6854.67,7203.04,4385.672853,95.01,95.01,0.00,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-03,7203.04,7401.20,7176.07,7334.45,9593.744578,131.41,131.41,0.00,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-04,7333.95,7398.00,7263.32,7348.63,3788.097227,14.18,14.18,0.00,NaN,NaN,NaN,NaN,0,0,0,0,0
2020-01-05,7355.40,7580.00,7343.17,7528.70,3362.380611,180.07,180.07,0.00,NaN,NaN,NaN,NaN,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-26,44433.22,44973.50,44156.22,44601.24,7235.396772,167.92,167.92,0.00,629.586429,238.700000,2.637564,72.509072,-1,0,0,0,0
2022-03-27,44598.44,47694.00,44437.22,47148.98,12278.601793,2547.74,2547.74,0.00,811.567857,196.307143,4.134174,80.522670,-1,0,0,0,0
2022-03-28,47149.30,48240.00,46770.70,47624.51,17868.795255,475.53,475.53,0.00,827.055000,196.307143,4.213066,80.817432,-1,0,0,0,0
2022-03-29,47624.33,48124.94,46544.89,47255.62,15656.635480,-368.89,0.00,-368.89,803.036429,222.656429,3.606617,78.292095,-1,0,0,0,0


In [32]:
updated_df = df.merge(rsi_df[['RSI_Signal']], left_on=df.index, right_on=rsi_df[['RSI_Signal']].index, how='left')
updated_df.rename({'key_0': 'datetime'}, inplace=True, axis=1)
updated_df.set_index('datetime', inplace=True)
updated_df.sort_index(ascending=True, inplace=True)
updated_df.head()

,Fuzzy_sentiment_signal,MACRO_0,MACRO_1,MACRO_2,MA_Signal,VaR,Open,High,Low,Close,Volume,Bollinger,RSI_Signal
datetime,,,,,,,,,,,,,
2021-01-01,0,3661.709111,-5.580458,0.925199,0.0,3111,29413.29,29683.50,29039.00,29683.03,4431.837894,0,0
2021-01-02,0,3757.358041,-5.094828,1.771221,0.0,3546,32222.88,34205.00,32008.62,33779.48,7955.017558,-1,0
2021-01-03,0,3853.006971,-4.609197,2.617242,0.0,3450,33082.83,33666.99,32453.37,32903.87,7401.195215,0,0
2021-01-04,0,3948.655902,-4.123567,3.463264,0.0,3187,32020.22,32890.00,30020.00,30460.19,11908.475812,0,0
2021-01-05,0,3780.935518,-3.040439,1.718721,0.0,3698,34043.91,35848.99,33352.54,35130.05,11648.279116,-1,0


In [33]:
updated_df.to_csv('IS5006_Historical.csv')